In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install openai==0.28.0

In [ ]:
!pip install faiss-gpu

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
import re
from tqdm import tqdm
import openai

In [ ]:
# Configure sua chave da API da OpenAI
openai.api_key = ''

In [ ]:
# Carregar os dados da planilha
file_path = '/content/cpp_t1 - cleaned_cpp_posts_t2.csv'
df = pd.read_csv(file_path)


In [ ]:
# Usar apenas as colunas relevantes (PostId, Title, Body)
df = df[['PostId', 'Title', 'Body']]

In [ ]:
# Carregar o modelo de embedding
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Função para gerar embeddings em lotes
def batch_encode(texts, batch_size=64):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch, show_progress_bar=False)
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# Combinar o título e o corpo
texts = (df['Title'] + ' ' + df['Body']).tolist()

# Criar embeddings por lotes
embeddings = batch_encode(texts)

100%|██████████| 439/439 [52:49<00:00,  7.22s/it]


In [ ]:
# Verifique a dimensionalidade dos embeddings
d = embeddings.shape[1]

In [ ]:
# Criar um índice de similaridade usando FAISS (distância L2)
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings, dtype=np.float32))  # Adiciona os embeddings ao índice



---



*PARA TESTE APENAS - UTILIZANDO CALCULO COSENAL - COMPARAR COM QDRANT*

In [ ]:
# Normaliza os embeddings para usar a similaridade cosenal
#embeddings_normalized = np.array(embeddings, dtype=np.float32)
#faiss.normalize_L2(embeddings_normalized)

# Cria o índice com a métrica de distância L2 (mas já com os vetores normalizados)
#index = faiss.IndexFlatIP(d)  # Inner Product é usado para similaridade cosenal
#index.add(embeddings_normalized)  # Adiciona os embeddings ao índice



---



In [ ]:
def retrieve(query, k=5):
    # Gerar embedding da consulta
    query_embedding = model.encode([query])

    # Buscar os K textos mais similares
    D, I = index.search(np.array(query_embedding, dtype=np.float32), k)

    # Retornar os textos recuperados (com base no índice)
    return df.iloc[I[0]]

In [ ]:
# Função para filtrar posts relevantes (baseado em termos da pergunta) com priorização de palavras-chave
def filter_relevant_posts(posts, query):
    # Extrair palavras-chave da query
    keywords = [re.escape(word) for word in query.lower().split()]

    # Filtrar posts que contenham os termos da query
    filtered_posts = posts[posts['Body'].str.contains('|'.join(keywords), case=False, na=False)]

    # Se nenhum post relevante for encontrado, retornar todos os posts recuperados
    return filtered_posts if not filtered_posts.empty else posts

In [ ]:
# Função para dividir o contexto em partes menores
def split_context(context, max_len=500):
    sentences = context.split('. ')
    chunks = []
    chunk = ''

    for sentence in sentences:
        if len(chunk) + len(sentence) + 1 <= max_len:
            chunk += sentence + '. '
        else:
            chunks.append(chunk.strip())
            chunk = sentence + '. '

    if chunk:
        chunks.append(chunk.strip())

    return chunks

    # Imprimir os chunks para verificação
    for i, c in enumerate(chunks):
        print(f"Chunk {i}: {c[:100]}...")  # Exibir apenas uma parte do chunk

    return chunks

In [ ]:
# Função aprimorada para gerar respostas
def generate_answer(query, k=5):
    # Recuperar e filtrar posts mais relevantes
    retrieved_posts = retrieve(query, k=k)
    filtered_posts = filter_relevant_posts(retrieved_posts, query)

    # Concatenar os textos dos posts filtrados
    context = " ".join(filtered_posts['Body'].values)

    # Verificar se o contexto está vazio ou muito curto
    if not context.strip() or len(context.split()) < 10:
        return "Nenhuma informação relevante encontrada para a pergunta."

    # Dividir o contexto em partes menores
    context_chunks = split_context(context)

    # Iterar sobre os chunks e usar o modelo OpenAI para gerar respostas
    answers = []
    for chunk in context_chunks:
        if chunk.strip():  # Verificar se o chunk não está vazio
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",  # ou "gpt-4"
                messages=[
                    {"role": "system", "content": "Você é um assistente útil que responde perguntas com base em contexto."},
                    {"role": "user", "content": f"Pergunta: {query}\nContexto: {chunk}"}
                ]
            )
            answers.append(response['choices'][0]['message']['content'].strip())

    # Combinar as respostas e remover duplicatas
    final_answer = '. '.join(set(answers))

    return final_answer if final_answer else "Nenhuma resposta clara foi encontrada."

In [ ]:
# Testando a consulta
query = "o que é c++?"
response = generate_answer(query)
print(f"Pergunta: {query}")
print(f"Resposta: {response}")

Pergunta: o que é c++?
Resposta: C++ é uma linguagem de programação de propósito geral que foi derivada de C. Ela oferece recursos adicionais e suporte para programação orientada a objetos, bem como outras funcionalidades avançadas, como templates e exceções. Em relação à sua pergunta, uma diferença chave entre C e C++ é o suporte a programação orientada a objetos e outras características específicas de C++. Assim, mesmo que muitas coisas possam ser feitas em ambas as linguagens, certas funcionalidades específicas a C++ poderiam não estar disponíveis ou ser mais complicadas de implementar em C.

Quanto ao exemplo de stack overflow em C++, esse problema ocorre quando a pilha de execução do programa atinge seu limite de capacidade devido ao grande número de chamadas de função aninhadas. Isso pode acontecer, por exemplo, se uma função chama a si mesma repetidamente sem parar. Nesse caso, a pilha de execução se enche rapidamente, resultando em um estouro de pilha (stack overflow) e causand

In [ ]:
# Testando a consulta
query = "What are the main benefits and challenges of migrating from C++11 to C++14?"
response = generate_answer(query)
print(f"Pergunta: {query}")
print(f"Resposta: {response}")

Pergunta: What are the main benefits and challenges of migrating from C++11 to C++14?
Resposta: Migrating from C++11 to C++14 can bring several benefits and challenges. 

Benefits:
1. Enhanced language features: C++14 introduces new features and improvements over C++11, such as simplified generic programming with variable templates, improved type deduction with decltype(auto), and more.
2. Performance improvements: Code written in C++14 may be more optimized and efficient compared to C++11, leading to potential performance gains.
3. Compatibility: C++14 is backward-compatible with C++11, so existing C++11 codebases can generally be upgraded to C++14 with minimal changes.

Challenges:
1. Retesting: As mentioned, migrating to C++14 would require retesting the code on all deployed servers to ensure compatibility and stability.
2. Toolchain support: Depending on the development tools and libraries in use, there may be a need to update or adapt them for C++14 compatibility.
3. STL implement

In [ ]:
# Testando a consulta
query = "Can you list StackOverflow questions related to migrating from C++11 to C++14?"
response = generate_answer(query)
print(f"Pergunta: {query}")
print(f"Resposta: {response}")

Pergunta: Can you list StackOverflow questions related to migrating from C++11 to C++14?
Resposta: Para listar perguntas relacionadas à migração de C++11 para C++14 no StackOverflow, você pode pesquisar por termos como "migrating from C++11 to C++14" na barra de pesquisa do site. É provável que você encontre várias perguntas e respostas sobre esse tópico específico.

No geral, ao migrar de C++11 para C++14, você pode esperar encontrar mudanças principalmente nos recursos de linguagem e bibliotecas disponíveis. Algumas das diferenças notáveis podem incluir melhorias de desempenho, novos recursos de linguagem e aprimoramentos nas bibliotecas padrão.

Em relação ao uso do Microsoft Visual Studio e do GCC para compilar seu código C++, ambos os compiladores oferecem suporte para C++14, mas pode haver diferenças de implementação entre eles. Recomenda-se testar a compilação e execução do código em ambos os compiladores para garantir a compatibilidade.

Como seu software atualmente suporta Win